In [54]:
import pandas as pd

In [55]:
ikea_dataset = pd.read_csv('ikea.csv', index_col='item_id')
ikea_dataset = ikea_dataset.drop(columns=['Unnamed: 0', 'sellable_online', 'old_price'])

In [56]:
# ikea_dataset = ikea_dataset.iloc[5:10]

In [57]:
link_list = list(zip(ikea_dataset['link'],ikea_dataset['link'].index))

In [58]:
ikea_dataset

,name,category,price,link,other_colors,short_description,designer,depth,height,width
item_id,,,,,,,,,,
90420332,FREKVENS,Bar furniture,265.0,https://www.ikea.com/sa/en/p/frekvens-bar-tabl...,No,"Bar table, in/outdoor, 51x51 cm",Nicholai Wiig Hansen,NaN,99.0,51.0
368814,NORDVIKEN,Bar furniture,995.0,https://www.ikea.com/sa/en/p/nordviken-bar-tab...,No,"Bar table, 140x80 cm",Francis Cayouette,NaN,105.0,80.0
9333523,NORDVIKEN / NORDVIKEN,Bar furniture,2095.0,https://www.ikea.com/sa/en/p/nordviken-nordvik...,No,Bar table and 4 bar stools,Francis Cayouette,NaN,NaN,NaN
80155205,STIG,Bar furniture,69.0,https://www.ikea.com/sa/en/p/stig-bar-stool-wi...,Yes,"Bar stool with backrest, 74 cm",Henrik Preutz,50.0,100.0,60.0
30180504,NORBERG,Bar furniture,225.0,https://www.ikea.com/sa/en/p/norberg-wall-moun...,No,"Wall-mounted drop-leaf table, ...",Marcus Arvonen,60.0,43.0,74.0
...,...,...,...,...,...,...,...,...,...,...
99157902,ELVARLI,Wardrobes,750.0,https://www.ikea.com/sa/en/p/elvarli-1-section...,No,"1 section, 92x51x222-350 cm",Ehlén Johansson,50.0,NaN,91.0
9158152,ELVARLI,Wardrobes,1572.0,https://www.ikea.com/sa/en/p/elvarli-2-section...,No,"2 sections, 135x51x222-350 cm",Ehlén Johansson,50.0,NaN,135.0
59157541,ELVARLI,Wardrobes,924.0,https://www.ikea.com/sa/en/p/elvarli-2-section...,No,"2 sections, 175x51x222-350 cm",Ehlén Johansson,50.0,NaN,175.0


In [59]:
link_list

[('https://www.ikea.com/sa/en/p/frekvens-bar-table-in-outdoor-black-90420332/',
  90420332),
 ('https://www.ikea.com/sa/en/p/nordviken-bar-table-black-00368814/', 368814),
 ('https://www.ikea.com/sa/en/p/nordviken-nordviken-bar-table-and-4-bar-stools-black-black-s09333523/',
  9333523),
 ('https://www.ikea.com/sa/en/p/stig-bar-stool-with-backrest-black-silver-colour-80155205/',
  80155205),
 ('https://www.ikea.com/sa/en/p/norberg-wall-mounted-drop-leaf-table-white-30180504/',
  30180504),
 ('https://www.ikea.com/sa/en/p/ingolf-bar-stool-with-backrest-white-10122647/',
  10122647),
 ('https://www.ikea.com/sa/en/p/franklin-bar-stool-with-backrest-foldable-white-white-70404875/',
  70404875),
 ('https://www.ikea.com/sa/en/p/dalfred-bar-stool-black-60155602/', 60155602),
 ('https://www.ikea.com/sa/en/p/franklin-bar-stool-with-backrest-foldable-black-black-50406465/',
  50406465),
 ('https://www.ikea.com/sa/en/p/ekedalen-ekedalen-bar-table-and-4-bar-stools-dark-brown-orrsta-light-grey-s6930

In [60]:
import requests
from bs4 import BeautifulSoup

def get_url_href_results(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

def check_product_page_availability(search_term):
    return get_url_href_results(search_term)

You can stop this process whenever you feel like you have enough IKEA products to add to your database! It can take a long time to web scrape and verify all of the images from all of the IKEA products.

In [61]:
valid_product_id_list = []
valid_product_image_list = []

for link, index in link_list:
    soup = check_product_page_availability(link)

    for div in soup.find_all(id="pip-media-1"):
        image_link = div.find('img')
        print("Found the Image:", image_link["src"])
        image_link = image_link["src"]
        
        valid_product_id_list.append(index)
        valid_product_image_list.append(image_link)
        
print((valid_product_id_list, valid_product_image_list))

Found the Image: https://www.ikea.com/sa/en/images/products/norberg-wall-mounted-drop-leaf-table-white__0736622_pe740674_s5.jpg?f=s
Found the Image: https://www.ikea.com/sa/en/images/products/ingolf-bar-stool-with-backrest-white__0728062_pe736035_s5.jpg?f=s
Found the Image: https://www.ikea.com/sa/en/images/products/franklin-bar-stool-with-backrest-foldable-white-white__0727489_pe735714_s5.jpg?f=s
Found the Image: https://www.ikea.com/sa/en/images/products/dalfred-bar-stool-black__0727486_pe735711_s5.jpg?f=s
Found the Image: https://www.ikea.com/sa/en/images/products/franklin-bar-stool-with-backrest-foldable-black-black__0727485_pe735710_s5.jpg?f=s
Found the Image: https://www.ikea.com/sa/en/images/products/franklin-bar-stool-with-backrest-foldable-white-white__0727493_pe735718_s5.jpg?f=s
Found the Image: https://www.ikea.com/sa/en/images/products/norraryd-bar-stool-with-backrest-black__0728070_pe736043_s5.jpg?f=s
Found the Image: https://www.ikea.com/sa/en/images/products/tommaryd-tab

KeyboardInterrupt: 

In [62]:
valid_product_image_list

['https://www.ikea.com/sa/en/images/products/norberg-wall-mounted-drop-leaf-table-white__0736622_pe740674_s5.jpg?f=s',
 'https://www.ikea.com/sa/en/images/products/ingolf-bar-stool-with-backrest-white__0728062_pe736035_s5.jpg?f=s',
 'https://www.ikea.com/sa/en/images/products/franklin-bar-stool-with-backrest-foldable-white-white__0727489_pe735714_s5.jpg?f=s',
 'https://www.ikea.com/sa/en/images/products/dalfred-bar-stool-black__0727486_pe735711_s5.jpg?f=s',
 'https://www.ikea.com/sa/en/images/products/franklin-bar-stool-with-backrest-foldable-black-black__0727485_pe735710_s5.jpg?f=s',
 'https://www.ikea.com/sa/en/images/products/franklin-bar-stool-with-backrest-foldable-white-white__0727493_pe735718_s5.jpg?f=s',
 'https://www.ikea.com/sa/en/images/products/norraryd-bar-stool-with-backrest-black__0728070_pe736043_s5.jpg?f=s',
 'https://www.ikea.com/sa/en/images/products/tommaryd-table-anthracite__0740544_pe742053_s5.jpg?f=s',
 'https://www.ikea.com/sa/en/images/products/janinge-bar-stoo

In [66]:
len(valid_product_id_list)

355

In [67]:
len(valid_product_image_list)

355

In [91]:
working_ikea_dataset = ikea_dataset.loc[valid_product_id_list]
working_ikea_dataset = working_ikea_dataset[~working_ikea_dataset.index.duplicated()]

In [92]:
working_ikea_dataset

,name,category,price,link,other_colors,short_description,designer,depth,height,width
item_id,,,,,,,,,,
30180504,NORBERG,Bar furniture,225.0,https://www.ikea.com/sa/en/p/norberg-wall-moun...,No,"Wall-mounted drop-leaf table, ...",Marcus Arvonen,60.0,43.0,74.0
10122647,INGOLF,Bar furniture,345.0,https://www.ikea.com/sa/en/p/ingolf-bar-stool-...,No,"Bar stool with backrest, 63 cm",Carina Bengs,45.0,91.0,40.0
70404875,FRANKLIN,Bar furniture,129.0,https://www.ikea.com/sa/en/p/franklin-bar-stoo...,No,"Bar stool with backrest, foldable, ...",K Hagberg/M Hagberg,44.0,95.0,50.0
60155602,DALFRED,Bar furniture,195.0,https://www.ikea.com/sa/en/p/dalfred-bar-stool...,No,"Bar stool, 63-74 cm",Sarah Fager,50.0,NaN,50.0
50406465,FRANKLIN,Bar furniture,129.0,https://www.ikea.com/sa/en/p/franklin-bar-stoo...,No,"Bar stool with backrest, foldable, ...",K Hagberg/M Hagberg,44.0,95.0,50.0
...,...,...,...,...,...,...,...,...,...,...
30357003,KALLRÖR,Bookcases & shelving units,45.0,https://www.ikea.com/sa/en/p/kallroer-handle-s...,No,"Handle, 597 mm",IKEA of Sweden,38.0,NaN,NaN
40247368,BESTÅ,Bookcases & shelving units,195.0,https://www.ikea.com/sa/en/p/besta-frame-grey-...,Yes,"Frame, 60x40x128 cm",IKEA of Sweden,40.0,128.0,60.0
70388639,HAVSTA,Bookcases & shelving units,545.0,https://www.ikea.com/sa/en/p/havsta-shelving-u...,Yes,"Shelving unit, 81x35x123 cm",IKEA of Sweden,35.0,123.0,81.0


In [93]:
working_ikea_dataset['link'] = valid_product_image_list

In [ ]:
def force_show_max_width(df):
    with pd.option_context('display.max_colwidth', None, 'display.width', None):
        display(df)

In [107]:
working_ikea_dataset['category'].unique()

array(['Bar furniture', 'Beds', 'Bookcases & shelving units'],
      dtype=object)

In [108]:
working_ikea_dataset['category'] = working_ikea_dataset['category'].replace({'Bar furniture': 1, 'Beds': 2, 'Bookcases & shelving units': 3})

In [110]:
working_ikea_dataset.index = working_ikea_dataset.index.rename('product_id')

In [115]:
working_ikea_dataset = working_ikea_dataset.rename(columns={'category': 'category_id'})

In [117]:
working_ikea_dataset = working_ikea_dataset.rename(columns={'link': 'image'})

In [121]:
working_ikea_dataset['other_colors'] = working_ikea_dataset['other_colors'].replace({'Yes': True, 'No': False})

In [122]:
force_show_max_width(working_ikea_dataset)

,name,category_id,price,image,other_colors,short_description,designer,depth,height,width
product_id,,,,,,,,,,
30180504,NORBERG,1,225.0,https://www.ikea.com/sa/en/images/products/norberg-wall-mounted-drop-leaf-table-white__0736622_pe740674_s5.jpg?f=s,False,"Wall-mounted drop-leaf table, 74x60 cm",Marcus Arvonen,60.0,43.0,74.0
10122647,INGOLF,1,345.0,https://www.ikea.com/sa/en/images/products/ingolf-bar-stool-with-backrest-white__0728062_pe736035_s5.jpg?f=s,False,"Bar stool with backrest, 63 cm",Carina Bengs,45.0,91.0,40.0
70404875,FRANKLIN,1,129.0,https://www.ikea.com/sa/en/images/products/franklin-bar-stool-with-backrest-foldable-white-white__0727489_pe735714_s5.jpg?f=s,False,"Bar stool with backrest, foldable, 63 cm",K Hagberg/M Hagberg,44.0,95.0,50.0
60155602,DALFRED,1,195.0,https://www.ikea.com/sa/en/images/products/dalfred-bar-stool-black__0727486_pe735711_s5.jpg?f=s,False,"Bar stool, 63-74 cm",Sarah Fager,50.0,NaN,50.0
50406465,FRANKLIN,1,129.0,https://www.ikea.com/sa/en/images/products/franklin-bar-stool-with-backrest-foldable-black-black__0727485_pe735710_s5.jpg?f=s,False,"Bar stool with backrest, foldable, 63 cm",K Hagberg/M Hagberg,44.0,95.0,50.0
...,...,...,...,...,...,...,...,...,...,...
30357003,KALLRÖR,3,45.0,https://www.ikea.com/sa/en/images/products/kallroer-handle-stainless-steel__0602016_pe680087_s5.jpg?f=s,False,"Handle, 597 mm",IKEA of Sweden,38.0,NaN,NaN
40247368,BESTÅ,3,195.0,https://www.ikea.com/sa/en/images/products/besta-frame-grey-stained-walnut-effect__0316150_pe513527_s5.jpg?f=s,True,"Frame, 60x40x128 cm",IKEA of Sweden,40.0,128.0,60.0
70388639,HAVSTA,3,545.0,https://www.ikea.com/sa/en/images/products/havsta-shelving-unit-white__0820439_pe732420_s5.jpg?f=s,True,"Shelving unit, 81x35x123 cm",IKEA of Sweden,35.0,123.0,81.0


In [97]:
import mysql.connector

ikea_connection = mysql.connector.connect(user='admin', password='ikeasolutionsarchitect',
                              host='revisiting-pep-i-ikea.c5lobpudlayi.us-west-1.rds.amazonaws.com',
                              database='ikea_products')

In [101]:
from sqlalchemy import create_engine

In [104]:
import pymysql

In [ ]:
import dbconn

db_username = dbconn.db_username
db_password = dbconn.db_password
db_host = dbconn.db_host
db_name = dbconn.db_name

db_url = f"mysql+pymysql://{db_username}:{db_password}@{db_host}/{db_name}"
db_connection = create_engine(db_connection_str)

In [132]:
working_ikea_dataset.to_sql(name='products', con=db_connection, if_exists='append')

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '30180504' for key 'products.PRIMARY'")
[SQL: INSERT INTO products (product_id, name, category_id, price, image, other_colors, short_description, designer, depth, height, width) VALUES (%(product_id)s, %(name)s, %(category_id)s, %(price)s, %(image)s, %(other_colors)s, %(short_description)s, %(designer)s, %(depth)s, %(height)s, %(width)s)]
[parameters: [{'product_id': 30180504, 'name': 'NORBERG', 'category_id': 1, 'price': 225.0, 'image': 'https://www.ikea.com/sa/en/images/products/norberg-wall-mounted-drop-leaf-table-white__0736622_pe740674_s5.jpg?f=s', 'other_colors': 0, 'short_description': '        Wall-mounted drop-leaf table,          74x60 cm', 'designer': 'Marcus Arvonen', 'depth': 60.0, 'height': 43.0, 'width': 74.0}, {'product_id': 10122647, 'name': 'INGOLF', 'category_id': 1, 'price': 345.0, 'image': 'https://www.ikea.com/sa/en/images/products/ingolf-bar-stool-with-backrest-white__0728062_pe736035_s5.jpg?f=s

In [130]:
working_ikea_dataset.to_json('ikea_products.json')